# Set-UID Programs and Environment Variables

## Need for Privileged Programs

- many system programs and configuration files are root/admin owned with restricted access to regular users
- some of these programs need to be executed by regular users regular usage of the system

## Password Dilemma

- let's see the permission of `/etc/shadow` file
- how do regular users change their password?

In [1]:
! ls -al /etc/shadow

-rw-r----- 1 root shadow 1504 Jan 17 13:37 /etc/shadow


In [2]:
! getfacl /etc/shadow

getfacl: Removing leading '/' from absolute path names
# file: etc/shadow
# owner: root
# group: shadow
user::rw-
group::r--
other::---



## Two-Tier Approach

- implementing fine-grained access control in OS make the over complicated
    - requires major redesign/rewrite of all major OS kernels
- OS relies on extension to enforce fine grained access control
- privileged programs are such extensions

![image.png](./media/setuid-approach.png)

- there are two types of privileged programs:

### Daemons
- computer program/process that runs in the background
- typically needs to run as root or other privileged users


### Set-UID programs
- widely used in \*NIX systems
- programs are marked with a speical bit

## Setuid Programs

- setuid programs are a device to allow users to acquire new priviledges for a limited amunt of time
- they provide a meas for overriding the protection schemes designed in \*NIX systems
- `setuid` programs are written in a way that it provides a granual access to individual line of a protected file/resource
- powersuit analogy - suit that gives power to whoever uses it
- which of the following group is the power suit?
    - superman/spiderman suit?
    - ironman/antman suit?
    
- allow users to run a program with the program owner's privilege
- the privilege is temporary while the program is being executed

### Case study - passwd program
- **passwd** program is used to change user's password
- let's see the access control list of **passwd** program
- note `s` in ACL and the owner:group

In [3]:
! ls -l /usr/bin/passwd

-rwsr-xr-x 1 root root 72344 Jan  5 19:57 /usr/bin/passwd


In [4]:
! getfacl /usr/bin/passwd

getfacl: Removing leading '/' from absolute path names
# file: usr/bin/passwd
# owner: root
# group: root
# flags: s--
user::rwx
group::r-x
other::r-x



## Set-UID Concept

- every process has two User IDs:

#### Real UID (RUID)
- identifies the real owner of the process

#### Effective UID (EUID)
- identifies privilege of a process
- access control is based on **EUID**

- when a normal program is executed, **RUID = EUID**
    - both IDs equal to the ID of the user who runs the program
- when a **set-uid** is executed, **RUID != EUID**
    - RUID still equals to user's ID who runs the program
    - EUID howover is equal to the program owner's ID
    - if the program is owned by root, the program runs with the root privilege
    
### meowcat - copy cat
- create a program that behaves like the `cat` program
- the demo program is provided in `demos/setuid` folder
- let's change the working diectory to `demos/setuid`
- use jupyter notebook's magic symbol to do that: `%`

In [1]:
%pwd

'/home/kali/projects/SoftwareSecurity'

In [2]:
%cd demos/setuid 

/home/kali/projects/SoftwareSecurity/demos/setuid


In [10]:
# if you get a warning when using magic symbol...
# UserWarning: using dhist requires you to install the `pickleshare` library.
! pip install pickleshare

In [4]:
! ls -al

total 56
drwxr-xr-x  2 kali kali  4096 Jan 17 21:08 .
drwxr-xr-x 14 kali kali  4096 Jan 17 20:44 ..
-rw-r--r--  1 kali kali  1014 Jan 17 21:08 Makefile
-rwxr-xr-x  1 kali kali 37160 Jan 17 21:08 meowcat
-rw-r--r--  1 kali kali   892 Jan 17 21:07 meowcat.cpp


In [9]:
! cat meowcat.cpp

// Program that mimics bash cat program
#include <iostream>
#include <cstdio>
#include <fstream>
#include <string>

using namespace std;

int main(int argc, char* argv[]) {
    if (argc <= 1) {
        printf("Usage: program <filepath>\n");
        return 1;
    }
    ifstream fin;
    fin.open(argv[1], fin.in | fin.binary | fin.ate);

    if (!fin.is_open())
        cout << "Failed to open " << argv[1] << '\n';
    else {
        // findout the size of the the file; get position in input sequence
        size_t size = fin.tellg();
        // Set position in input sequence
        fin.seekg(0, fin.beg);
        // allocate memory to store file contents
        char * buffer = new char[size];
        
        if (fin.read(buffer, size))
        {
            cout << buffer << endl;
            // parse buffer in memory...
        }
        delete[] buffer;
        fin.close();
    }
    return 0;
}


In [10]:
! cat Makefile

# Makefile Template
# variable for compiler name
COMPILER = g++

# c++ compiler flags
# -c : just do preprocess; compile and assemble into object files
# -g : adds debugging information to the executable file
# -Wall : turns on most, but not  all, compiler warnings
# -std=c++17 : uses c++17 standard
# put all the required flags separated by space
COMPILER_FLAGS = -g -Wall -std=c++17

# the build target executable program name of your choice
PROGRAM_NAME = meowcat # FIXME

# list all .cpp files separated by space; files must exist!
CPP_FILES = meowcat.cpp # FIXME

# make build rule/default rule that'll be run when we run "make" or make build
# first compile each file to object file
# then make a single binary program
build:
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES) -o $(PROGRAM_NAME)
	# sudo chown root:root $(PROGRAM_NAME)
	# sudo chmod +s $(PROGRAM_NAME)

# make clean rule
# delete object and binary files
clean:
	rm -f $(PROGRAM_NAME) *.o


In [5]:
! ./meowcat

Usage: program <filepath>


In [6]:
! echo "hello there..." > data.txt

In [8]:
! ./meowcat data.txt

hello there...



In [11]:
! ./meowcat /etc/shadow

Failed to open /etc/shadow


In [12]:
# let's set the ownership of the program to root to setuid
! echo kali | sudo -S chown root meowcat

[sudo] password for kali: 

In [13]:
! ls -al meowcat

-rwxr-xr-x 1 root kali 37160 Jan 17 21:27 meowcat


In [14]:
# let's setuid chmod 4755
! echo kali | sudo -S chmod +s meowcat

[sudo] password for kali: 

In [15]:
! ls -al meowcat

-rwsr-sr-x 1 root kali 37160 Jan 17 21:27 meowcat


In [17]:
! ./meowcat /etc/shadow

root:!:19662:0:99999:7:::
daemon:*:19662:0:99999:7:::
bin:*:19662:0:99999:7:::
sys:*:19662:0:99999:7:::
sync:*:19662:0:99999:7:::
games:*:19662:0:99999:7:::
man:*:19662:0:99999:7:::
lp:*:19662:0:99999:7:::
mail:*:19662:0:99999:7:::
news:*:19662:0:99999:7:::
uucp:*:19662:0:99999:7:::
proxy:*:19662:0:99999:7:::
www-data:*:19662:0:99999:7:::
backup:*:19662:0:99999:7:::
list:*:19662:0:99999:7:::
irc:*:19662:0:99999:7:::
_apt:*:19662:0:99999:7:::
nobody:*:19662:0:99999:7:::
systemd-network:!*:19662::::::
_galera:!:19662::::::
mysql:!:19662::::::
tss:!:19662::::::
strongswan:!:19662::::::
systemd-timesync:!*:19662::::::
redsocks:!:19662::::::
rwhod:!:19662::::::
_gophish:!:19662::::::
iodine:!:19662::::::
messagebus:!:19662::::::
miredo:!:19662::::::
redis:!:19662::::::
usbmux:!:19662::::::
mosquitto:!:19662::::::
tcpdump:!:19662::::::
sshd:!:19662::::::
_rpc:!:19662::::::
dnsmasq:!:19662::::::
statd:!:19662::::::
avahi:!:19662::::::
stunnel4:!*:19662::::::
Debian-snmp:!:19662::::::
_gvm:!:1

In [19]:
# if the owner isn't root or privileged user, the program is setuid but NOT privilege
! echo kali | sudo -S chown bob meowcat

[sudo] password for kali: 

In [20]:
! ls -al meowcat

-rwxr-xr-x 1 bob kali 37160 Jan 17 21:27 meowcat


In [21]:
! echo kali | sudo -S chmod 4777 meowcat

[sudo] password for kali: 

In [22]:
! ls -al meowcat

-rwsrwxrwx 1 bob kali 37160 Jan 17 21:27 meowcat


In [23]:
! ./meowcat /etc/shadow

Failed to open /etc/shadow


## What programs need setuid?

- programs that require to access sensitive, system-wide data, files, configs, etc.
- is it safe to setuid every programs?
    - /bin/cat, /bin/sh, nano, vi, etc.


## Environment Variables

- environment variables are integral part of all OS
- these are dynamic values that can have significant influence over the behavior of programs and processes
- these variables are outside the program that store configuration or settings and their values can be changed without modifying the programs' actual code 
- one of the most important `env` variable is `PATH`
    - PATH stores the locations to search for various binaries, libraries, and executable programs
- programs and scripts can access environment variables to adapt their behavior based on the configuration and requirements of the system they're being executed
- OSes provide various ways to set and access environment variables
- system APIs are available to programming languages to access and set environment variables in the programs
- in Linux, env variables play important roles when using Terminals to run programs
- `env` or `printenv` commands list all the environment variables available to the current user
- variable names are case sensitive
- normally ALL_CAPS are used for env variable names

In [24]:
! env

COLORFGBG=15;0
COLORTERM=truecolor
COMMAND_NOT_FOUND_INSTALL_PROMPT=1
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus
DESKTOP_SESSION=lightdm-xsession
DISPLAY=:0.0
DOTNET_CLI_TELEMETRY_OPTOUT=1
GDMSESSION=lightdm-xsession
GTK_MODULES=gail:atk-bridge
HOME=/home/kali
LANG=C.UTF-8
LANGUAGE=
LOGNAME=kali
PANEL_GDK_CORE_DEVICE_EVENTS=0
PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games
POWERSHELL_TELEMETRY_OPTOUT=1
POWERSHELL_UPDATECHECK=Off
PWD=/home/kali/projects/SoftwareSecurity/demos/setuid
QT_ACCESSIBILITY=1
QT_AUTO_SCREEN_SCALE_FACTOR=0
QT_QPA_PLATFORMTHEME=qt5ct
SESSION_MANAGER=local/kali:@/tmp/.ICE-unix/1090,unix/kali:/tmp/.ICE-unix/1090
SHELL=/usr/bin/zsh
SSH_AGENT_PID=1142
SSH_AUTH_SOCK=/tmp/ssh-TxkQPF1pUHj8/agent.1090
TERM=xterm-color
USER=kali
WINDOWID=0
XAUTHORITY=/home/kali/.Xauthority
XDG_CONFIG_DIRS=/etc/xdg
XDG_CURRENT_DESKTOP=XFCE
XDG_DATA_DIRS=/usr/share/xfce4:/usr/lo

### Setting and Accessing Environment Variables

- env variables can be set in many places depending on the needs

1. System wide
    - /etc/environment
    - /etc/profile
2. User specific
    - ~/.profile
    - ~/.bashrc
    - ~/.zshrc

In [25]:
! cat /etc/environment

# START KALI-DEFAULTS CONFIG
# Everything from here and until STOP KALI-DEFAULTS CONFIG
# was installed by the kali-defaults package, and it will
# be removed if ever the kali-defaults package is removed.
# If you want to disable a line, please do NOT remove it,
# as it would be added back when kali-defaults is upgraded.
# Instead, comment the line out, and your change will be
# preserved across upgrades.
PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games
COMMAND_NOT_FOUND_INSTALL_PROMPT=1
POWERSHELL_UPDATECHECK=Off
POWERSHELL_TELEMETRY_OPTOUT=1
DOTNET_CLI_TELEMETRY_OPTOUT=1
# STOP KALI-DEFAULTS CONFIG


In [26]:
! cat $HOME/.zshrc

# ~/.zshrc file for zsh interactive shells.
# see /usr/share/doc/zsh/examples/zshrc for examples

setopt autocd              # change directory just by typing its name
#setopt correct            # auto correct mistakes
setopt interactivecomments # allow comments in interactive mode
setopt magicequalsubst     # enable filename expansion for arguments of the form ‘anything=expression’
setopt nonomatch           # hide error message if there is no match for the pattern
setopt notify              # report the status of background jobs immediately
setopt numericglobsort     # sort filenames numerically when it makes sense
setopt promptsubst         # enable command substitution in prompt

WORDCHARS=${WORDCHARS//\/} # Don't consider certain characters part of the word

# hide EOL sign ('%')
PROMPT_EOL_MARK=""

# configure key keybindings
bindkey -e                                        # emacs key bindings
bindkey ' ' magic-space                           # do history expansion on space
bin

In [29]:
! echo $PATH

/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games


### Types of Env Variables

#### shell variable
- like local variables in programs; available to the current shell until it's terminated
- can't have trailing and leading space around the '=' sign unlike in many programming languages
- space is a special character -- word/command separater -- in bash scripting

```bash
VAR_NAME="VALUE..."
```

#### environment or system variables
- like global variables; they can be available system-wide across the Terminals
- temporary env variable can be exported from the Terminal and availble only to that Terminal
- permenant env variables are exported from init script such as `~/.zshrc` everytime a Terminal is launched

```bash
export VAR[="VALUE..."]
set VAR1="some other values"
```

- can't create shell and environemnt variables from Jupyter Notebooks
- use Terminal instead...
- add `$` before the variable name when reading the value

In [27]:
! VAR=test

In [28]:
! echo $VAR

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ VAR="Some data"
                                                                                                     
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ echo $VAR            
Some data

(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ cd             
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ echo $VAR
Some data

(base) ┌──(kali㉿kali)-[~]
└─$ export VAR
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ env | grep VAR
VAR=Some data
```

### Update existing env variable

- sometime we may need to temporarily and dynamically add a new path to the PATH variable

```bash
(base) ┌──(kali㉿kali)-[~]
└─$ PATH=$PATH:$HOME/bin
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ export PATH 

# one liner
(base) ┌──(kali㉿kali)-[~]
└─$ export PATH=$PATH:$HOME/bin

(base) ┌──(kali㉿kali)-[~]
└─$ env | grep ^PATH=
PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games:/home/kali//home/kali/bin
```

### Delete environment variables

- can use `unset` command or assign empty value to temporarily delete them
- can update the corresponding setting files to permanently delete them

```
unset NAME
NAME=""
```

## Futher Readings and References
- How to Write Setuid Programs - https://nob.cs.ucdavis.edu/~bishop/secprog/1987-sproglogin.pdf
- Setuid BSDI Man page - https://seedsecuritylabs.org/Labs_20.04/Software/Environment_Variable_and_SetUID/files/setuid.pdf
- Setuid Demystified - https://web.ecs.syr.edu/~wedu/minix/projects/setuid_paper.pdf